In [1]:
from os import listdir
from datetime import date
import numpy as np

In [6]:
allfilenames_set7 = ['raw_data/set7_ac/stdout/'+f for f in listdir('raw_data/set7_ac/stdout')]
allfilenames_set5 = ['raw_data/set5_ac/stdout/'+f for f in listdir('raw_data/set5_ac/stdout')]

In [3]:
def median_ASAQ_time_calculator(allfilenames):
  allAdpCycSwitchTimes = []
  for filename in allfilenames:
    if filename == 'parser.ipynb' or filename == '.ipynb_checkpoints': continue
    with open(filename) as f:
      lines = f.readlines()
    temp = []
    for info in lines:
      if 'Cycling Strategy Swith Therapy to:' in info:
        # yyyy, mm, dd, therapySwitchedTo
        therapySwitchedTo = info[-2:-1]
        yyyy, mm, dd = info[17:27].split('-')
        temp.append( (yyyy, mm, dd, therapySwitchedTo) )
    allAdpCycSwitchTimes.append(temp)
  
  allDaysASAQInUse = []
  for switchList in allAdpCycSwitchTimes:
    if len(switchList) == 0: # no switch, ASAQ not in use
      allDaysASAQInUse.append(0)
      continue
    if len(switchList) == 2: # switch twice, ASAQ used in between
      st, end = switchList
    else: # switched once, ASAQ used until end of simulation
      st = switchList[0]
      end = ('2040','1','1')
    d0 = date(int(st[0]), int(st[1]), int(st[2]))
    d1 = date(int(end[0]), int(end[1]), int(end[2]))
    delta = d1 - d0
    allDaysASAQInUse.append(delta.days / 365)
  
  assert(len(allDaysASAQInUse)==100)
  return np.percentile(allDaysASAQInUse, [25,50,75], interpolation = 'midpoint')

Set 7ac, Figure 2 in main paper

In [7]:
median_ASAQ_time_calculator(allfilenames_set7)

array([6.55205479, 8.00410959, 9.29178082])

Set 5ac, Figure 3 in main paper

In [8]:
median_ASAQ_time_calculator(allfilenames_set5)

array([0.32465753, 2.17123288, 5.3260274 ])